In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, precision_recall_curve, roc_auc_score
import seaborn
from matplotlib import pyplot as plt

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
df_train = pd.read_csv('/data/home/alexey.astafiev/lab04/laba04_train.csv')
df_test = pd.read_csv('/data/home/alexey.astafiev/lab04/laba04_test.csv')

In [3]:
y_target = df_train['TARGET']

In [4]:
cols = df_train.columns.values.tolist()

In [5]:
del cols[0]

In [6]:
del cols[len(cols)-1]


In [7]:
trainX = df_train[cols]
testX = df_test[cols]

In [8]:
fix_trainX = trainX.drop(['CNT_ACCEPTS_MTP', 'CNT_ACCEPTS_TK', 'PRC_ACCEPTS_A_AMOBILE', 'PRC_ACCEPTS_A_ATM', 'PRC_ACCEPTS_A_EMAIL_LINK', 'PRC_ACCEPTS_A_MTP', 'PRC_ACCEPTS_A_POS', 'PRC_ACCEPTS_A_TK', 'PRC_ACCEPTS_MTP', 'PRC_ACCEPTS_TK'], axis=1)
fix_testX = testX.drop(['CNT_ACCEPTS_MTP', 'CNT_ACCEPTS_TK', 'PRC_ACCEPTS_A_AMOBILE', 'PRC_ACCEPTS_A_ATM', 'PRC_ACCEPTS_A_EMAIL_LINK', 'PRC_ACCEPTS_A_MTP', 'PRC_ACCEPTS_A_POS', 'PRC_ACCEPTS_A_TK', 'PRC_ACCEPTS_MTP', 'PRC_ACCEPTS_TK'], axis=1)
fix_trainX = fix_trainX.drop(['CLNT_JOB_POSITION'], axis=1)
fix_testX = fix_testX.drop(['CLNT_JOB_POSITION'], axis=1)
fix_trainX[['CLNT_JOB_POSITION_TYPE', 'APP_POSITION_TYPE']] = fix_trainX[['CLNT_JOB_POSITION_TYPE', 'APP_POSITION_TYPE']].fillna('none')
fix_testX[['CLNT_JOB_POSITION_TYPE', 'APP_POSITION_TYPE']] = fix_testX[['CLNT_JOB_POSITION_TYPE', 'APP_POSITION_TYPE']].fillna('none')

In [9]:
def union_cols(x):
    if x[0] == 'none':
        if x[1] == 'none':
            return 'none'
        else:
            return x[1]
    elif x[0] == x[1]:
        return x[0]
    else:
        return x[0]
        

In [10]:
fix_trainX['CLNT_JOB_TYPE'] = fix_trainX[['CLNT_JOB_POSITION_TYPE', 'APP_POSITION_TYPE']].apply(union_cols, axis=1)
fix_testX['CLNT_JOB_TYPE'] = fix_testX[['CLNT_JOB_POSITION_TYPE', 'APP_POSITION_TYPE']].apply(union_cols, axis=1)

In [15]:
fix_trainX = fix_trainX.drop(['CLNT_JOB_POSITION_TYPE', 'APP_POSITION_TYPE'], axis=1)
fix_testX = fix_testX.drop(['CLNT_JOB_POSITION_TYPE', 'APP_POSITION_TYPE'], axis=1)

In [16]:
fix_trainX['CLNT_TRUST_RELATION'] = fix_trainX[['CLNT_TRUST_RELATION']].fillna('none')
fix_testX['CLNT_TRUST_RELATION'] = fix_testX[['CLNT_TRUST_RELATION']].fillna('none')

In [17]:
def f1(x):
#     if x[0] == 'none':
#         return 'OTHER'
#     elif unicode(x[0]) == u'Мать':
    if x[0].decode('utf-8') == u'Мать':
        return 'MOTHER'
    elif x[0].decode('utf-8') == u'Друг':
        return 'FRIEND'
    elif x[0].decode('utf-8') == u'Отец':
        return 'FATHER'
    elif x[0].decode('utf-8') == u'Брат':
        return 'BROTHER'
    elif x[0].decode('utf-8') == u'Близкий ро':
        return 'RELATIVE'
    elif x[0].decode('utf-8') == u'Сестра':
        return 'SISTER'
    elif x[0].decode('utf-8') == u'Дальний ро':
        return 'RELATIVE'
    elif x[0].decode('utf-8') == u'Сын':
        return 'SON'
    elif x[0].decode('utf-8') == u'Дочь':
        return 'DAUGHTER'
    elif x[0].decode('utf-8') == u'Жена':
        return 'WIFE'
    elif x[0].decode('utf-8') == u'Муж':
        return 'HUSBAND'
    elif x[0].decode('utf-8') == u'мать':
        return 'MOTHER'
    elif x[0].decode('utf-8') == u'мама':
        return 'MOTHER'
    else:
        return x[0]


In [18]:
fix_trainX['CLNT_TRUST_RELATION1'] = fix_trainX[['CLNT_TRUST_RELATION']].apply(f1, axis=1)
fix_testX['CLNT_TRUST_RELATION1'] = fix_testX[['CLNT_TRUST_RELATION']].apply(f1, axis=1)
fix_trainX = fix_trainX.drop(['CLNT_TRUST_RELATION'], axis=1)
fix_testX = fix_testX.drop(['CLNT_TRUST_RELATION'], axis=1)

In [19]:
# pickle.dump(fix_trainX, open('/data/home/alexey.astafiev/lab04/fix_trainX.pcl', 'w'))

In [20]:
for i in fix_trainX.columns.values:
    if fix_trainX[i].dtype == 'object':
        fix_trainX[i] = fix_trainX[i].fillna('none')

for i in fix_testX.columns.values:
    if fix_testX[i].dtype == 'object':
        fix_testX[i] = fix_testX[i].fillna('none')

In [21]:
def f2(x):
    if x[0] == 'v':
        return 'V'
    elif x[0] == 'm':
        return 'M'
    elif x[0] == 't':
        return 'T'
    elif x[0] == 'd':
        return 'D'
    elif x[0] == 'w':
        return 'W'
    else:
        return x[0]



In [22]:
fix_trainX['APP_MARITAL_STATUS'] = fix_trainX[['APP_MARITAL_STATUS']].apply(f2, axis=1)
fix_testX['APP_MARITAL_STATUS'] = fix_testX[['APP_MARITAL_STATUS']].apply(f2, axis=1)

In [23]:
for i in fix_trainX.columns.values:
    if fix_trainX[i].dtype == 'int64':
        fix_trainX[i] = fix_trainX[i].fillna(0)
    elif fix_trainX[i].dtype == 'float64':
        fix_trainX[i] = fix_trainX[i].fillna(0.0)

for i in fix_testX.columns.values:
    if fix_testX[i].dtype == 'int64':
        fix_testX[i] = fix_testX[i].fillna(0)
    elif fix_testX[i].dtype == 'float64':
        fix_testX[i] = fix_testX[i].fillna(0.0)

In [25]:
uids_train = fix_trainX['ID']
uids_test = fix_testX['ID']

In [26]:
all_data = pd.concat([fix_trainX, fix_testX], ignore_index=True)

In [27]:
categ_cols = []
for i in fix_trainX.columns.values:
    if fix_trainX[i].dtype == 'object':
        categ_cols.append(i)

In [28]:
print categ_cols

['APP_MARITAL_STATUS', 'APP_KIND_OF_PROP_HABITATION', 'APP_DRIVING_LICENSE', 'APP_EDUCATION', 'APP_TRAVEL_PASS', 'APP_CAR', 'APP_EMP_TYPE', 'APP_COMP_TYPE', 'PACK', 'CLNT_JOB_TYPE', 'CLNT_TRUST_RELATION1']


In [29]:
X = pd.get_dummies(all_data, columns=categ_cols)

In [30]:
X_train = X[X['ID'].isin(uids_train.values.tolist())]
X_train = X_train.drop(['ID'], axis=1)

In [31]:
X_test = X[X['ID'].isin(uids_test.values.tolist())]
X_test = X_test.drop(['ID'], axis=1)

In [32]:
y_test = X_test.index.values

In [33]:
model = RandomForestClassifier(n_estimators=100, n_jobs=-1, oob_score = True)

In [34]:
model.fit(X_train, y_target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)

In [36]:
predictions = model.predict_proba(X_test)

In [41]:
with open('/data/home/alexey.astafiev/lab04.csv', 'w') as fout:
    fout.write("id\ttarget\n")
    for uid, pred in zip(y_test, predictions[:, 1]):
        fout.write("%s\t%s\n" % (uid, pred))